In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/loadmodel/keras/default/1/model_8814_42.h5
/kaggle/input/test-data/positions_processed.csv
/kaggle/input/test-data-set/test_track_8.csv
/kaggle/input/training-data/Chess_1.csv
/kaggle/input/training-data/Chess_3.csv
/kaggle/input/training-data/Chess_2.csv


## scope for improvement

* add king score depnding on the game pahase
* train the current model on chess reduced
  



##  sumbission entry:

1. (first submission) submission2.csv --> used model 8814_42, trained Chess1.csv, main epoch = 3

## Import necessary modules

In [3]:
import pandas as pd
import numpy as np
import tensorflow
import sklearn

## Some board pre processing functions 

In [4]:
piece_to_col = {
    'P':0, 'B':1,'N':2,'R':3,'Q':4,'K':5,
    'p':6, 'b':7,'n':8,'r':9,'q':10,'k':11,
}

col_to_piece = {
    val:key for key, val in piece_to_col.items()
}
def getBoard(fen):
    fen_str = fen.split(' ')[0]
    ranks = fen_str.split('/')
    board = []
    for i in range(8):
        rank = ranks[i]
        board_rank = ""
        for j in rank:
            if(j.isdigit()):
                board_rank += '.'*int(j)

            else:
                board_rank += j
        board.append(board_rank)
    return board

# print(getBoard("r2q1rk1/pb3pbp/1ppp1np1/2nPp3/4P3/2P1B2P/PPBNNPP1/R2Q1RK1 w - - 0 12"))

In [5]:
def convert_fen(fen):
    ls = fen.split(' ')

    if(ls[1] == 'w'): #activeColor = 'w'
        return fen

    #if active color is 'b' then we need to convert it to 'w'
    fen_str = ls[0]
    fen_str = fen_str[::-1]
    new_fen_str = ""

    for char in fen_str:
        if(char == '/' or (char.isdigit())):
            new_fen_str += char
        else:
            if(char.islower()):
                new_fen_str += char.upper()
            else:
                new_fen_str += char.lower()

    new_fen_ls = new_fen_str.split('/')
    new_fen_tmp = ''
    for rank in new_fen_ls :
        new_fen_tmp += rank[::-1]
        new_fen_tmp += '/'

    new_fen_str = new_fen_tmp[:-1]

    cas_str = ls[2]

    new_cas_str = ""

    for char in cas_str:
        if(char == '-'):
            new_cas_str += char
        else:
            if(char.islower()):
                new_cas_str += char.upper()
            else:
                new_cas_str += char.lower()


    cas_str1 = ""

    if(new_cas_str == "-"):
      cas_str1 = "-"

    else:

      if('K' in new_cas_str):
        cas_str1 += "K"

      if("Q" in new_cas_str):
        cas_str1 += "Q"

      if('k' in new_cas_str):
        cas_str1 += "k"
      if("q" in new_cas_str):
        cas_str1 += "q"



    ep = ls[3]
    if(ep != '-') :
        c2 = int(ep[1]) - 1
        c2 = 8 - c2
        ep = ep[:-1]
        ep += str(c2)


    new_ls = [new_fen_str, 'w', cas_str1, ep ,ls[4], ls[5]]
    new_fen = ""
    for i in range(5):
        new_fen += new_ls[i]
        new_fen += " "
    new_fen += new_ls[5]

    return new_fen

In [6]:

piece_to_col = {
    'P':0, 'B':1,'N':2,'R':3,'Q':4,'K':5,
    'p':6, 'b':7,'n':8,'r':9,'q':10,'k':11,
}

def getWhitePawnMoves(board, x, y):
    moves = []

    if x > 0 and board[x - 1][y] == '.':
        moves.append([x - 1, y])

        if x == 6 and board[x - 2][y] == '.':
            moves.append([x - 2, y])

    if x > 0 and y > 0 and board[x - 1][y - 1].islower():
        moves.append([x - 1, y - 1])

    if x > 0 and y < 7 and board[x - 1][y + 1].islower():
        moves.append([x - 1, y + 1])

    return moves


def getBlackPawnMoves(board, x, y):
    moves = []

   
    if x < 7 and board[x + 1][y] == '.':
        moves.append([x + 1, y])

        
        if x == 1 and board[x + 2][y] == '.':
            moves.append([x + 2, y])

    
    if x < 7 and y > 0 and board[x + 1][y - 1].isupper():
        moves.append([x + 1, y - 1])

   
    if x < 7 and y < 7 and board[x + 1][y + 1].isupper():
        moves.append([x + 1, y + 1])

    return moves


def getKnightMoves(board, x, y, is_white):
    moves = []
    directions = [
        (-2, -1), (-2, 1), (-1, -2), (-1, 2),
        (1, -2), (1, 2), (2, -1), (2, 1)
    ]

    for dx, dy in directions:
        new_x, new_y = x + dx, y + dy
        if 0 <= new_x < 8 and 0 <= new_y < 8:
            if board[new_x][new_y] == '.' or (is_white and board[new_x][new_y].islower()) or (not is_white and board[new_x][new_y].isupper()):
                moves.append([new_x, new_y])

    return moves


def getSlidingMoves(board, x, y, directions, is_white):
    moves = []

    for dx, dy in directions:
        new_x, new_y = x + dx, y + dy
        while 0 <= new_x < 8 and 0 <= new_y < 8:
            if board[new_x][new_y] == '.':
                moves.append([new_x, new_y])
            elif (is_white and board[new_x][new_y].islower()) or (not is_white and board[new_x][new_y].isupper()):
                moves.append([new_x, new_y])
                break
            else:
                break
            new_x += dx
            new_y += dy

    return moves


def getBishopMoves(board, x, y, is_white):
    return getSlidingMoves(board, x, y, [(-1, -1), (-1, 1), (1, -1), (1, 1)], is_white)


def getRookMoves(board, x, y, is_white):
    return getSlidingMoves(board, x, y, [(-1, 0), (1, 0), (0, -1), (0, 1)], is_white)


def getQueenMoves(board, x, y, is_white):
    return getSlidingMoves(board, x, y,
        [(-1, -1), (-1, 1), (1, -1), (1, 1), (-1, 0), (1, 0), (0, -1), (0, 1)], is_white)


def isSquareAttacked(board, x, y, is_white):
    """Check if the square (x, y) is attacked by an opponent."""
    for i in range(8):
        for j in range(8):
            piece = board[i][j]
            if piece == '.':
                continue
            if (is_white and piece.islower()) or (not is_white and piece.isupper()):
                if piece.lower() == 'p':  # Check pawn attacks
                    if is_white and (i == x + 1 and (j == y + 1 or j == y - 1)):
                        return True
                    if not is_white and (i == x - 1 and (j == y + 1 or j == y - 1)):
                        return True
                elif piece.lower() == 'n':  
                    if [x, y] in getKnightMoves(board, i, j, not is_white):
                        return True
                elif piece.lower() == 'b': 
                    if [x, y] in getBishopMoves(board, i, j, not is_white):
                        return True
                elif piece.lower() == 'r':  
                    if [x, y] in getRookMoves(board, i, j, not is_white):
                        return True
                elif piece.lower() == 'q':  
                    if [x, y] in getQueenMoves(board, i, j, not is_white):
                        return True
                elif piece.lower() == 'k': 
                    if abs(i - x) <= 1 and abs(j - y) <= 1:
                        return True
    return False


def getKingMoves(board, x, y, is_white):
    moves = []
    directions = [
        (-1, -1), (-1, 0), (-1, 1), (0, -1),
        (0, 1), (1, -1), (1, 0), (1, 1)
    ]

    for dx, dy in directions:
        new_x, new_y = x + dx, y + dy
        if 0 <= new_x < 8 and 0 <= new_y < 8:
            if board[new_x][new_y] == '.' or (is_white and board[new_x][new_y].islower()) or (not is_white and board[new_x][new_y].isupper()):
                if not isSquareAttacked(board, new_x, new_y, is_white):  # King cannot move into check
                    moves.append([new_x, new_y])

    return moves


def getMoves(board , piece , x , y) :
    moves = []
    is_white = True
    if(piece.islower()) :
        is_white = False

    if(piece == 'P') :
        moves = getWhitePawnMoves(board , x , y)
    elif(piece == 'p') :
        moves = getBlackPawnMoves(board, x, y)
    elif(piece == 'N' or piece == 'n') :
        moves = getKnightMoves(board, x, y, is_white)
    elif(piece == 'B' or piece == 'b') :
        moves = getBishopMoves(board, x, y, is_white)
    elif(piece == 'R' or piece == 'r') :
        moves = getRookMoves(board, x, y, is_white)
    elif(piece == 'Q' or piece == 'q') :
        moves = getQueenMoves(board, x, y, is_white)
    elif(piece == 'K' or piece == 'k') :
        moves = getKingMoves(board, x, y, is_white)

    return moves



def getAttackingSquares(board) :
    white_attack = np.zeros((8 , 8))
    black_attack = np.zeros((8 , 8))

    for i in range(8) :
        for j in range(8) :
            piece = board[i][j]
            if(piece == '.') :
                continue
            if(piece.isupper()) :
                moves = getMoves(board , piece , i , j)
                for move in moves :
                    white_attack[move[0]][move[1]] = 1

            else :
                moves = getMoves(board , piece , i , j)
                for move in moves :
                    black_attack[move[0]][move[1]] = 1

    return white_attack , black_attack


## Encoding chess board into 8x8x14

In [7]:
def encoding8814(fen):

    fen_str = fen.split(' ')[0]
    board_encoding = np.zeros((8,8,14))
    # print(board_encoding)
    board = getBoard(fen)
    # print(board)

    for i in range(8):
        row = board[i]
        # print(row)
        for j in range(8):
            piece = row[j]
            # print(piece)
            if(piece != '.'):
                board_encoding[i][j][piece_to_col[piece]] = 1


    white_legal, black_legal = getAttackingSquares(board)

    board_encoding[:,:,12] = white_legal
    board_encoding[:,:,13] = black_legal

    return board_encoding

In [8]:
def active_color(fen):
    color = fen.split(' ')[1]
    return color

def flip_eval(row):
    fen = row['FEN']
    color = active_color(fen)
    if(color == 'w'):
        return row['Evaluation']
    else:
        return -row['Evaluation']

def processData(df):
    df['Evaluation'] = df.apply(flip_eval, axis = 1)
    df['FEN'] = df['FEN'].apply(convert_fen)
    return df

## Additional features

In [9]:
def rookFeatures(board):
  '''
    return [count of rooks on 7th rank, number of open files controlled by rook,
    rook battery, mobility]
  '''
  white_rook_count_on_7th_rank = 0
  white_open_files_controlled = 0
  white_semi_open_files_controlled = 0
  white_mobility = 0

  black_rook_count_on_7th_rank = 0
  black_open_files_controlled = 0
  black_semi_open_files_controlled = 0
  black_mobility = 0

  white_battery = 0
  black_battery = 0

  white_rook_positions = []
  black_rook_positions = []

  for i in range(8):
    for j in range(8):
      if(board[i][j] == 'R'):
        white_rook_positions.append([i,j])
      if(board[i][j] == 'r'):
        black_rook_positions.append([i,j])

  # print(white_rook_positions)

  for position in white_rook_positions:

    if(position[0] == 1):
      white_rook_count_on_7th_rank += 1

    occupied_by_white_pawn = False
    occupied_by_black_pawn = False
    i = position[0] #rank
    j = position[1] #file
    # print("white rook at",i,j)
    legal_moves = getRookMoves(board, i, j, 1)
    # print(legal_moves)
    white_mobility += len(legal_moves)

    for rank in range(8):
      if(rank != i):
        if(board[rank][j] == 'P'):
          occupied_by_white_pawn = True
        if(board[rank][j] == 'p'):
          occupied_by_black_pawn = True

    # print(occupied_by_white_pawn)
    # print(occupied_by_black_pawn)

    if((not occupied_by_white_pawn) and (not occupied_by_black_pawn)):
        white_open_files_controlled += 1
    if((not occupied_by_white_pawn) and (occupied_by_black_pawn)):
        white_semi_open_files_controlled += 1

  for position in black_rook_positions:

      if(position[0] == 6):
        black_rook_count_on_7th_rank += 1

      occupied_by_white_pawn = False
      occupied_by_black_pawn = False
      i = position[0] #rank
      j = position[1] #file

      legal_moves = getRookMoves(board, i, j, 0)
      black_mobility += len(legal_moves)

      for rank in range(8):
        if(rank != i):
          if(board[rank][j] == 'P'):
            occupied_by_white_pawn = True
          if(board[rank][j] == 'p'):
            occupied_by_black_pawn = True

      if((not occupied_by_white_pawn) and (not occupied_by_black_pawn)):
        black_open_files_controlled += 1
      if((not occupied_by_black_pawn) and (occupied_by_white_pawn)):
        black_semi_open_files_controlled += 1


  if(len(white_rook_positions) == 2):
    rook1_i = white_rook_positions[0][0]
    rook1_j = white_rook_positions[0][1]

    rook2_i = white_rook_positions[1][0]
    rook2_j = white_rook_positions[1][1]

    white_battery = True

    if(rook1_i == rook2_i):
      min_file = min(rook1_j, rook2_j)
      max_file = max(rook1_j, rook2_j)
      for j in range(min_file+1, max_file):
        if(board[rook1_i][j] != '.'):
          white_battery = False


    if(rook1_j == rook2_j):
      min_rank = min(rook1_i, rook2_i)
      max_rank = max(rook1_i, rook2_i)
      for i in range(min_rank+1, max_rank):
        if(board[i][rook1_j] != '.'):
          white_battery = False


  if(len(black_rook_positions) == 2):
    rook1_i = black_rook_positions[0][0]
    rook1_j = black_rook_positions[0][1]

    rook2_i = black_rook_positions[1][0]
    rook2_j = black_rook_positions[1][1]


    black_battery = True

    if(rook1_i == rook2_i):

      min_file = min(rook1_j, rook2_j)
      max_file = max(rook1_j, rook2_j)
      for j in range(min_file+1, max_file):
        # print(board[rook1_i][j])
        if(board[rook1_i][j] != '.'):
          black_battery = False
    # print(black_battery)

    if(rook1_j == rook2_j):
      min_rank = min(rook1_i, rook2_i)
      max_rank = max(rook1_i, rook2_i)
      for i in range(min_rank+1, max_rank):
        if(board[i][rook1_j] != '.'):
          black_battery = False



  return [
      [white_rook_count_on_7th_rank, white_open_files_controlled, white_semi_open_files_controlled, white_battery, white_mobility],
      [black_rook_count_on_7th_rank, black_open_files_controlled, black_semi_open_files_controlled, black_battery, black_mobility]

  ]



In [10]:
def BishopFeatures(board):

  '''
    [bishop pair, bishop in center,  mobility] for white and black
  '''
  white_bishop_positions = []
  black_bishop_positions = []

  for rank in range(8):
    for file in range(8):
      piece = board[rank][file]
      if(piece == 'B'):
        white_bishop_positions.append([rank, file])
      if(piece == 'b'):
        black_bishop_positions.append([rank, file])

  white_mobility = 0
  black_mobility = 0
  white_bishop_pair = 0
  black_bishop_pair = 0
  white_bishop_on_center = 0
  black_bishop_on_center = 0

  if(len(white_bishop_positions) >= 2):
    white_bishop_pair = 1
  if(len(black_bishop_positions) >= 2):
    black_bishop_pair = 1

  center_squares = [[3,3], [3,4], [4,3], [4,4]]

  for position in white_bishop_positions:
    legal_moves = getBishopMoves(board, position[0], position[1], 1)
    white_mobility += len(legal_moves)
    if(position in center_squares):
      white_bishop_on_center += 1


  for position in black_bishop_positions:
    legal_moves = getBishopMoves(board, position[0], position[1], 0)
    black_mobility += len(legal_moves)
    if(position in center_squares):
      black_bishop_on_center += 1


  return [
      [white_bishop_pair, white_bishop_on_center, white_mobility],
      [black_bishop_pair, black_bishop_on_center, black_mobility]
  ]


In [11]:
def KnightFeatures(board):
  '''
    [knight on center, mobility] for white and black
  '''

  center_squares = [[3,3], [3,4], [4,3], [4,4]]

  white_knight_on_center = 0
  black_knight_on_center = 0

  white_mobility = 0
  black_mobility = 0

  white_knight_outpost = 0
  black_knight_outpost = 0

  white_knight_positions = []
  black_knight_positions = []

  for rank in range(8):
    for file in range(8):
      if(board[rank][file] == 'N'):
        white_knight_positions.append([rank, file])
      if(board[rank][file] == 'n'):
        black_knight_positions.append([rank, file])

  for position in white_knight_positions:
    if(position in center_squares):
      white_knight_on_center += 1
    legal_moves = getKnightMoves(board, position[0], position[1], 1)
    white_mobility += len(legal_moves)

  for position in black_knight_positions:
    if(position in center_squares):
      black_knight_on_center += 1
    legal_moves = getKnightMoves(board, position[0], position[1], 0)
    black_mobility += len(legal_moves)

  white_rank_low = 1
  white_rank_high = 4
  white_file_low = 2
  white_file_high = 5

  black_rank_high = 6
  black_rank_low = 3
  black_file_low = 2
  black_file_high = 5


  for position in white_knight_positions:
    rank = position[0]
    file = position[1]
    ok = (white_rank_low <= rank and rank <= white_rank_high) and (white_file_low <= file and file <= white_file_high)
    if(not ok):
      continue


    defended = False
    if(board[rank+1][file-1] == 'P' or board[rank+1][file + 1] == 'P'):
      defended = True


    can_be_attacked = False

    for j in range(rank-1,0, -1):
      if(board[j][file-1] == 'p' or board[j][file+1] == 'p'):
        can_be_attacked = True

     

    if(defended and not can_be_attacked):
      white_knight_outpost += 1

  for position in black_knight_positions:
    rank = position[0]
    file = position[1]
    ok = (black_rank_low <= rank and rank <= black_rank_high) and (black_file_low <= file and file <= black_file_high)

    if(not ok):
      continue


    defended = False
    if(board[rank-1][file-1] == 'p' or board[rank-1][file + 1] == 'p'):
      defended = True


    can_be_attacked = False

    for j in range(rank+1,7):
      if(board[j][file-1] == 'P' or board[j][file+1] == 'P'):
        can_be_attacked = True

    if(defended and (not can_be_attacked)):
      black_knight_outpost += 1


  return[
      [white_knight_on_center, white_knight_outpost, white_mobility],
      [black_knight_on_center,black_knight_outpost, black_mobility]
  ]







In [12]:
PAWN_TABLE = [
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [50, 50, 50, 50, 50, 50, 50, 50],
    [10, 10, 20, 30, 30, 20, 10, 10],
    [ 5,  5, 10, 25, 25, 10,  5,  5],
    [ 0,  0,  0, 20, 20,  0,  0,  0],
    [ 5, -5,-10,  0,  0,-10, -5,  5],
    [ 5, 10, 10,-20,-20, 10, 10,  5],
    [ 0,  0,  0,  0,  0,  0,  0,  0]
]

KNIGHT_TABLE = [
    [-50, -40, -30, -30, -30, -30, -40, -50],
    [-40, -20,   0,   0,   0,   0, -20, -40],
    [-30,   0,  10,  15,  15,  10,   0, -30],
    [-30,   5,  15,  20,  20,  15,   5, -30],
    [-30,   0,  15,  20,  20,  15,   0, -30],
    [-30,   5,  10,  15,  15,  10,   5, -30],
    [-40, -20,   0,   5,   5,   0, -20, -40],
    [-50, -40, -30, -30, -30, -30, -40, -50]
]

BISHOP_TABLE = [
    [-20, -10, -10, -10, -10, -10, -10, -20],
    [-10,   0,   0,   0,   0,   0,   0, -10],
    [-10,   0,   5,  10,  10,   5,   0, -10],
    [-10,   5,   5,  10,  10,   5,   5, -10],
    [-10,   0,  10,  10,  10,  10,   0, -10],
    [-10,  10,  10,  10,  10,  10,  10, -10],
    [-10,   5,   0,   0,   0,   0,   5, -10],
    [-20, -10, -10, -10, -10, -10, -10, -20]
]

ROOK_TABLE = [
    [  0,   0,   0,   0,   0,   0,   0,   0],
    [  5,  10,  10,  10,  10,  10,  10,   5],
    [ -5,   0,   0,   0,   0,   0,   0,  -5],
    [ -5,   0,   0,   0,   0,   0,   0,  -5],
    [ -5,   0,   0,   0,   0,   0,   0,  -5],
    [ -5,   0,   0,   0,   0,   0,   0,  -5],
    [ -5,   0,   0,   0,   0,   0,   0,  -5],
    [  0,   0,   0,   5,   5,   0,   0,   0]
]

QUEEN_TABLE = [
    [-20, -10, -10,  -5,  -5, -10, -10, -20],
    [-10,   0,   0,   0,   0,   0,   0, -10],
    [-10,   0,   5,   5,   5,   5,   0, -10],
    [ -5,   0,   5,   5,   5,   5,   0,  -5],
    [  0,   0,   5,   5,   5,   5,   0,  -5],
    [-10,   5,   5,   5,   5,   5,   0, -10],
    [-10,   0,   5,   0,   0,   0,   0, -10],
    [-20, -10, -10,  -5,  -5, -10, -10, -20]
]

KING_MIDDLE_GAME_TABLE = [
    [-30, -40, -40, -50, -50, -40, -40, -30],
    [-30, -40, -40, -50, -50, -40, -40, -30],
    [-30, -40, -40, -50, -50, -40, -40, -30],
    [-30, -40, -40, -50, -50, -40, -40, -30],
    [-20, -30, -30, -40, -40, -30, -30, -20],
    [-10, -20, -20, -20, -20, -20, -20, -10],
    [ 20,  20,   0,   0,   0,   0,  20,  20],
    [ 20,  30,  10,   0,   0,  10,  30,  20]
]


KING_END_GAME_TABLE = [
    [-50, -40, -30, -20, -20, -30, -40, -50],
    [-30, -20, -10,   0,   0, -10, -20, -30],
    [-30, -10,  20,  30,  30,  20, -10, -30],
    [-30, -10,  30,  40,  40,  30, -10, -30],
    [-30, -10,  30,  40,  40,  30, -10, -30],
    [-30, -10,  20,  30,  30,  20, -10, -30],
    [-30, -30,   0,   0,   0,   0, -30, -30],
    [-50, -30, -30, -30, -30, -30, -30, -50]
]

psqt_map_middlegame = {'P':PAWN_TABLE,
                       'N':KNIGHT_TABLE,
                       'B':BISHOP_TABLE,
                       'R':ROOK_TABLE,
                       'Q':QUEEN_TABLE,
                       'K':KING_MIDDLE_GAME_TABLE}

psqt_map_endgame = {'P':PAWN_TABLE,
                       'N':KNIGHT_TABLE,
                       'B':BISHOP_TABLE,
                       'R':ROOK_TABLE,
                       'Q':QUEEN_TABLE,
                       'K':KING_END_GAME_TABLE}

In [13]:

piece_to_value = {
    'P':100, 'p':-100, 'B':330, 'b':-330, 'N':320, 'n':-320, 'R':500, 'r':-500,
    'Q':900, 'q':-900, 'K':50, 'k':-50, '.':0
}

def value(peice):
    return piece_to_value[peice]

def is_endgame_fen(fen):
    WHITE_COUNT = 0
    BLACK_COUNT = 0
    fen_str = fen.split(' ')[0]
    board = fen_str

    for char in board:
      if(char == 'p' or char == 'P' or char == 'k' or char == 'K' or char == '/' or char.isdigit()):
        continue
      if(char.isupper()) :
        WHITE_COUNT += value(char)
      else :
        BLACK_COUNT += value(char)


    return abs(BLACK_COUNT) <= 1350

def is_endgame(board):
    WHITE_COUNT = 0
    BLACK_COUNT = 0

    for i in range(8):
      for char in board[i]:
        if(char == 'p' or char == 'P' or char == 'k' or char == 'K' or char == '/' or char.isdigit()):
          continue
        if(char.isupper()) :
          WHITE_COUNT += value(char)
        else :
          BLACK_COUNT += value(char)

    return abs(BLACK_COUNT) <= 1350


In [14]:
def pawn_structure_features(board) :
    '''
        return [double_isolated , double , isolated , central , backward]
    '''

    double_isolated_white , double_isolated_black = 0 , 0
    double_white , double_black = 0 , 0
    isolated_white , isolated_black = 0 , 0
    central_white , central_black = 0 , 0
    back_white , back_black = 0 , 0

    def get_count_in_column(board , piece , j) :
        if(j < 0 or j >= 8) :
            return 0

        count = 0
        for i in range(8) :
            if(board[i][j] == piece) :
                count += 1
        return count

    white_pawn_count = [0] * 8
    black_pawn_count = [0] * 8

    for j in range(8) :
        white_pawn_count[j] = get_count_in_column(board , 'P' , j)
        black_pawn_count[j] = get_count_in_column(board , 'p' , j)

    for j in range(8) :
        white_prev = 0
        if(j > 0) :
            white_prev = white_pawn_count[j - 1]
        white_next = 0
        if(j < 7) :
            white_next = white_pawn_count[j + 1]
        white_count = white_pawn_count[j]

        black_prev = 0
        if(j > 0) :
            black_prev = black_pawn_count[j - 1]
        black_next = 0
        if(j < 7) :
            black_next = black_pawn_count[j + 1]
        black_count = black_pawn_count[j]

        if(white_count > 0) :
            if(white_prev == 0 and white_next == 0 and white_count > 1) :
                double_isolated_white += (white_count - 1)
            elif(white_count > 1) :
                double_white += (white_count - 1)
            elif(white_prev == 0 and white_next == 0) :
                isolated_white += 1

        if(black_count > 0) :
            if(black_prev == 0 and black_next == 0 and black_count > 1) :
                double_isolated_black += (black_count - 1)
            elif(black_count > 1) :
                double_black += (black_count - 1)
            elif(black_prev == 0 and black_next == 0) :
                isolated_black += 1

    if(board[3][3] == 'P') :
        central_white += 1
    if(board[4][3] == 'P') :
        central_white += 1
    if(board[3][4] == 'P') :
        central_white += 1
    if(board[4][4] == 'P') :
        central_white += 1

    if(board[3][3] == 'p') :
        central_black += 1
    if(board[4][3] == 'p') :
        central_black += 1
    if(board[3][4] == 'p') :
        central_black += 1
    if(board[4][4] == 'p') :
        central_black += 1

    white_rank = [8] * 8
    black_rank = [8] * 8
    for j in range(8) :
        for i in range(8) :
            if(board[i][j] == 'p') :
                black_rank[j] = i
                break

        for i in range(8) :
            if(board[7 - i][j] == 'P') :
                white_rank[j] = i
                break

    for j in range(8) :
        if(white_rank[j] < 8) :
            prev_rank = 8
            next_rank = 8
            if(j > 0) :
                prev_rank = white_rank[j - 1]
            if(j < 7) :
                next_rank = white_rank[j + 1]

            if(white_rank[j] < prev_rank and white_rank[j] < next_rank) :
                back_white += 1

        if(black_rank[j] < 8) :
            prev_rank = 8
            next_rank = 8
            if(j > 0) :
                prev_rank = black_rank[j - 1]
            if(j < 7) :
                next_rank = black_rank[j + 1]

            if(black_rank[j] < prev_rank and black_rank[j] < next_rank) :
                back_black += 1

    back_white -= (isolated_white + double_isolated_white)
    back_black -= (isolated_black + double_isolated_black)

    white_queen , black_queen = 0 , 0
    white_king , black_king = 0 , 0

    for j in range(4) :
        for i in range(8) :
            if(board[i][j] == 'P') :
                white_queen += 1
            elif(board[i][j] == 'p') :
                black_queen += 1

            if(board[i][7 - j] == 'P') :
                white_king += 1
            elif(board[i][7 - j] == 'p') :
                black_king += 1



    return [
        [double_isolated_white, double_white, central_white, isolated_white, back_white, white_queen, white_king],
        [double_isolated_black, double_black, central_black, isolated_black, back_black, black_queen, black_king]
    ]


In [15]:
def is_passed_pawn(board , x, y):
    is_black = board[x][y] == 'p'
    ''' Check if a pawn at (y, x) is a passed pawn. '''
    direction = 1 if is_black else - 1
    for dy in [-1, 0, 1]:
        ny = y + dy
        if 0 <= ny < 8:
            for nx in range(x + direction, 8 if is_black else -1, direction):
                if((is_black and board[nx][ny] == 'P') or ((not is_black) and board[nx][ny] == 'p')) :
                    return False
    return True

def pass_pawn_features(board):
    '''
    Return [connected_pp, sum of advanced squares] for both white and black.

    Passed pawns are pawns with no opposing pawns in their file or adjacent files blocking their advancement.
    Connected passed pawns are passed pawns that are adjacent to each other.
    The sum of advanced squares is the total number of ranks each passed pawn has advanced from its starting position.
    '''
    white_connected, black_connected = 0, 0
    white_promotion, black_promotion = 0, 0
    white_passed, black_passed = [], []

    for x in range(8):
        for y in range(8):
            if board[x][y] == 'P':  
                if is_passed_pawn(board , x, y):
              
                    white_passed.append((x, y))
                    white_promotion += (7 - x) 
            elif board[x][y] == 'p': 
                if is_passed_pawn(board , x, y):
                    # print(x , y)
                    black_passed.append((x, y))
                    black_promotion += x  

    for x, y in white_passed:
        if (x, y - 1) in white_passed or (x, y + 1) in white_passed:
            white_connected += 1

    for x, y in black_passed:
        if (x, y - 1) in black_passed or (x, y + 1) in black_passed:
            black_connected += 1

   
    return [
          [white_connected, white_promotion],
          [black_connected, black_promotion]
    ]



In [16]:
def material_features(board) :
   
    psqt_map = psqt_map_middlegame
    if(is_endgame(board)) :
        psqt_map = psqt_map_endgame

    white_material , black_material = 0 , 0

    for i in range(8) :
        for j in range(8) :
            if(board[i][j] == '.') :
                continue
            abs_piece = board[i][j].upper()
            if(board[i][j].isupper()) :
                #white piece
                white_material += (value(abs_piece) + psqt_map[abs_piece][i][j])
            else :
                #black piece
                black_material += (value(abs_piece) + psqt_map[abs_piece][7-i][7-j])

    return [ [white_material] ,
            [black_material]]

In [17]:
def encode_additional_features(fen):
    board = getBoard(fen)
    features = (pawn_structure_features(board) + pass_pawn_features(board) + material_features(board) + rookFeatures(board) + BishopFeatures(board) + KnightFeatures(board))
    result = np.concatenate(features)
    return result

## Neural Network Model (CNN)

In [18]:

def build_deep_model():

  input = tensorflow.keras.layers.Input(shape = (8,8,14))

  input_additonal = tensorflow.keras.layers.Input(shape = (42,))

  x = tensorflow.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding = 'same', activation = 'relu')(input)
  z = tensorflow.keras.layers.Conv2D(filters=32, kernel_size=(4,4), padding='same', activation='relu')(input)
  y = tensorflow.keras.layers.Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu')(input)

  for _ in range(4):
      previous = x
      x = tensorflow.keras.layers.BatchNormalization()(x)
      x = tensorflow.keras.layers.Activation('relu')(x)
      x = tensorflow.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(x)
      x = tensorflow.keras.layers.BatchNormalization()(x)
      x = tensorflow.keras.layers.Add()([previous, x])
      x = tensorflow.keras.layers.Activation('relu')(x)
      x = tensorflow.keras.layers.Dropout(0.3)(x)

      previous2 = y
      y = tensorflow.keras.layers.BatchNormalization()(y)
      y = tensorflow.keras.layers.Activation('relu')(y)
      y = tensorflow.keras.layers.Conv2D(filters=32, kernel_size=(5, 5), padding='same', activation='relu')(y)
      y = tensorflow.keras.layers.BatchNormalization()(y)
      y = tensorflow.keras.layers.Add()([previous2, y])
      y = tensorflow.keras.layers.Activation('relu')(y)
      y = tensorflow.keras.layers.Dropout(0.3)(y)


      previous3 = z
      z = tensorflow.keras.layers.BatchNormalization()(z)
      z = tensorflow.keras.layers.Activation('relu')(z)
      z = tensorflow.keras.layers.Conv2D(filters=32, kernel_size=(4, 4), padding='same', activation='relu')(z)
      z = tensorflow.keras.layers.BatchNormalization()(z)
      z = tensorflow.keras.layers.Add()([previous3,z])
      z = tensorflow.keras.layers.Activation('relu')(z)
      z = tensorflow.keras.layers.Dropout(0.3)(z)


  x = tensorflow.keras.layers.Flatten()(x)
  y = tensorflow.keras.layers.Flatten()(y)
  z = tensorflow.keras.layers.Flatten()(z)

  cnn_features = tensorflow.keras.layers.Add()([x, y, z])
  x = tensorflow.keras.layers.Concatenate()([cnn_features, input_additonal])

  x = tensorflow.keras.layers.Dense(2090, activation = 'relu')(x)
  x = tensorflow.keras.layers.Dropout(0.3)(x)
  x = tensorflow.keras.layers.Dense(1)(x)

  model = tensorflow.keras.Model(inputs = [input, input_additonal], outputs = x)

  model.compile(optimizer = 'adam', loss = 'mse', metrics = ['mae'])

  model.summary()

  return model


## Load the traning data

In [19]:
data1 = pd.read_csv("/kaggle/input/training-data/Chess_1.csv")
data2 = pd.read_csv("/kaggle/input/training-data/Chess_2.csv")
chess_data_1 = processData(data1)
chess_data_2 = processData(data2)

In [20]:
# model1 = build_deep_model() # trained on chess1

In [21]:
model2 = build_deep_model() ## traning on chesss1 and chess2 

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 8, 8, 14)       │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 8, 8, 32)       │          4,064 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 8, 8, 32)       │         11,232 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 8, 8, 32)       │          7,200 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 8, 8, 32)       │            128 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 8, 8, 32)       │            128 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 8, 8, 32)       │            128 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 8, 8, 32)       │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 8, 8, 32)       │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_4 (Activation) │ (None, 8, 8, 32)       │              0 │ batch_normalization_4… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 8, 8, 32)       │          9,248 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 8, 8, 32)       │         25,632 │ activation_2[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 8, 8, 32)       │         16,416 │ activation_4[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 8, 8, 32)       │            128 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 8, 8, 32)       │            128 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_5     │ (None, 8, 8, 32)       │            128 │ conv2d_5[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)            

 Total params: 4,603,033 (17.56 MB)

 Trainable params: 4,601,497 (17.55 MB)

 Non-trainable params: 1,536 (6.00 KB)

In [ ]:
# training on chess1.csv and chess2.csv

data_splits = np.array_split(chess_data_2, 10)

for epoch in range(3):
    for i,data_chunk in enumerate(data_splits):

      train_data = data_chunk
      print(f"encoding chunk {i+1}/{len(data_splits)}..  (shape = {train_data.shape[0]})")

      y_train = train_data['Evaluation'].values
      encoded_fens_train = train_data['FEN'].apply(encoding8814)
      x_train = np.stack(encoded_fens_train.to_list())
      x_additional = np.stack(train_data['FEN'].apply(encode_additional_features).to_list())

      print("encoded!")
      print("training model...")
      # model1.fit(x_train, y_train, batch_size = 1024, validation_split=0.2, epochs = 10, verbose = 1)
      model2.fit([x_train, x_additional], y_train, batch_size=1024, validation_split=0.2, epochs=10, verbose=1)

print("trained on chess 2!")


data_splits = np.array_split(chess_data_1, 10)

for epoch in range(3):
    for i,data_chunk in enumerate(data_splits):

      train_data = data_chunk
      print(f"encoding chunk {i+1}/{len(data_splits)}..  (shape = {train_data.shape[0]})")

      y_train = train_data['Evaluation'].values
      encoded_fens_train = train_data['FEN'].apply(encoding8814)
      x_train = np.stack(encoded_fens_train.to_list())
      x_additional = np.stack(train_data['FEN'].apply(encode_additional_features).to_list())

      print("encoded!")
      print("training model...")
      # model1.fit(x_train, y_train, batch_size = 1024, validation_split=0.2, epochs = 10, verbose = 1)
      model2.fit([x_train, x_additional], y_train, batch_size=1024, validation_split=0.2, epochs=10, verbose=1)

print("trained on chess 1!")




/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


encoding chunk 1/10..  (shape = 198050)
encoded!
training model...
Epoch 1/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 55s 178ms/step - loss: 178133.3281 - mae: 244.7719 - val_loss: 209907.6875 - val_mae: 231.4301
Epoch 2/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 9s 58ms/step - loss: 80869.4219 - mae: 171.6637 - val_loss: 203834.2812 - val_mae: 248.7146
Epoch 3/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 9s 59ms/step - loss: 71030.4297 - mae: 159.9419 - val_loss: 133284.8438 - val_mae: 223.0594
Epoch 4/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 9s 59ms/step - loss: 61896.4023 - mae: 149.8292 - val_loss: 102365.8594 - val_mae: 211.3282
Epoch 5/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 9s 60ms/step - loss: 56438.9961 - mae: 143.6978 - val_loss: 75098.0547 - val_mae: 165.7335
Epoch 6/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 9s 60ms/step - loss: 50278.4922 - mae: 137.4396 - val_loss: 75491.3516 - val_mae: 160.3220
Epoch 7/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 9s 60ms/step - loss: 45395.6328 - mae: 132.6804 - val_loss: 66750.5781 - val_mae: 145.7946
Epoch 8/1

In [ ]:
# model1.save("model_8814_42.h5")  #model that was tranined only on chess1

In [24]:
 model2.save("model_8814_42_train21.h5") ## saves the model trained on chess1 and chess2

# testing the model

In [26]:
total_mse = 0
total_mae = 0

test_data = pd.read_csv("/kaggle/input/training-data/Chess_3.csv")
test_data = processData(test_data)

data_splits = np.array_split(test_data, 20)

model_loaded = tensorflow.keras.models.load_model("/kaggle/input/8814-on-chess2-and-chess1/keras/default/1/model_8814_42_train21.h5", custom_objects={'mse': tensorflow.keras.metrics.MeanSquaredError()})
print("model loaded!")

for index in range(len(data_splits)):
    test = data_splits[index]
    y_test = test['Evaluation'].values
    encoded_fens_train = test['FEN'].apply(encoding8814)
    x_test= np.stack(encoded_fens_train.to_list())
    print(f"loaded! size =({test.shape[0]})")
    x_additional = np.stack(test['FEN'].apply(encode_additional_features).to_list())
    loss, mae = model_loaded.evaluate([x_test, x_additional], y_test, verbose = 1)
    total_mse += loss*test.shape[0]
    total_mae += mae*test.shape[0]
    print(loss, mae)

total_mse /= test_data.shape[0]
total_mae /= test_data.shape[0]
print(f"mse = {total_mse}, mae = {total_mae}")

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


model loaded!
loaded! size =(99030)
3095/3095 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - loss: 36211.0703 - mean_absolute_error: 109.6359
38962.01171875 110.27685546875
loaded! size =(99030)
3095/3095 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 41195.8711 - mean_absolute_error: 111.4067
41393.57421875 111.10804748535156
loaded! size =(99030)
3095/3095 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 41595.8242 - mean_absolute_error: 112.5191
41690.390625 112.08338928222656
loaded! size =(99030)
3095/3095 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 41846.8594 - mean_absolute_error: 111.9308
41874.9765625 111.79689025878906
loaded! size =(99030)
3095/3095 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 42171.1211 - mean_absolute_error: 112.2683
42198.07421875 111.6853256225586
loaded! size =(99030)
3095/3095 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 42473.1797 - mean_absolute_error: 113.1800
42657.421875 112.67457580566406
loaded! size =(99029)
3095/3095 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 42782.1797 - me

In [ ]:
# total_mse = 0
# total_mae = 0

# test_data = pd.read_csv("/kaggle/input/training-data/Chess_3.csv")
# test_data = processData(test_data)

# data_splits = np.array_split(test_data, 20)

# model_loaded = tensorflow.keras.models.load_model("/kaggle/input/loadmodel/keras/default/1/model_8814_42.h5", custom_objects={'mse': tensorflow.keras.metrics.MeanSquaredError()})
# print("model loaded!")

# for index in range(len(data_splits)):
#     test = data_splits[index]
#     y_test = test['Evaluation'].values
#     encoded_fens_train = test['FEN'].apply(encoding8814)
#     x_test= np.stack(encoded_fens_train.to_list())
#     print(f"loaded! size =({test.shape[0]})")
#     x_additional = np.stack(test['FEN'].apply(encode_additional_features).to_list())
#     loss, mae = model_loaded.evaluate([x_test, x_additional], y_test, verbose = 1)
#     total_mse += loss*test.shape[0]
#     total_mae += mae*test.shape[0]
#     print(loss, mae)

# total_mse /= test_data.shape[0]
# total_mae /= test_data.shape[0]
# print(f"mse = {total_mse}, mae = {total_mae}")

## Making Submission!

In [27]:
import numpy as np
import pandas as pd
import tensorflow.keras.models


def processTest(test_data):
    test_data['FEN'] = test_data['FEN'].apply(convert_fen)
    return test_data


def makeSubmission(test_data, split_size):
    
    data_splits = np.array_split(test_data, split_size)

    predictions = []

    for index, test_chunk in enumerate(data_splits):
        
        print(f"loading chunk {index + 1}/{split_size} (size={test_chunk.shape[0]})")

        test_chunk = processTest(test_chunk)
        
        encoded_fens_test = test_chunk['FEN'].apply(encoding8814)
        x_test = np.stack(encoded_fens_test.to_list())

        x_additional = np.stack(test_chunk['FEN'].apply(encode_additional_features).to_list())
    
        chunk_predictions = model_loaded.predict([x_test, x_additional], verbose=1)

        print("predicting...")
        predictions.extend(chunk_predictions.flatten())
        
    return predictions



In [28]:
model_loaded = tensorflow.keras.models.load_model("/kaggle/input/8814-on-chess2-and-chess1/keras/default/1/model_8814_42_train21.h5", custom_objects={'mse': tensorflow.keras.metrics.MeanSquaredError()})
print("model loaded!")

test_data = pd.read_csv("/kaggle/input/test-data-set/test_track_8.csv")
test_data.rename(columns={'ID': 'FEN'}, inplace=True)

predictions = makeSubmission(test_data, 20)

sumbission_df = pd.DataFrame({"FEN": test_data["FEN"], "Evaluation": predictions})



model loaded!


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


loading chunk 1/20 (size=98970)
3093/3093 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step
predicting...
loading chunk 2/20 (size=98970)
3093/3093 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step
predicting...
loading chunk 3/20 (size=98970)
3093/3093 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step
predicting...
loading chunk 4/20 (size=98969)
3093/3093 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step
predicting...
loading chunk 5/20 (size=98969)
3093/3093 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step
predicting...
loading chunk 6/20 (size=98969)
3093/3093 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step
predicting...
loading chunk 7/20 (size=98969)
3093/3093 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step
predicting...
loading chunk 8/20 (size=98969)
3093/3093 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step
predicting...
loading chunk 9/20 (size=98969)
3093/3093 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step
predicting...
loading chunk 10/20 (size=98969)
3093/3093 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step
predicting...
loading chunk 11/20 (size=98969)
3093/3093 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step
predicting...
loading chunk 12/2

In [29]:
print(sumbission_df.head(10))

sumbission_df["Evaluation"] = sumbission_df.apply(flip_eval, axis=1)

sumbission_df.rename(columns={'FEN': 'ID'}, inplace=True)

print("\n\n")

print(sumbission_df.head(10))

sumbission_df.to_csv("submission3.csv", index=False)


                                                 FEN  Evaluation
0  r3k2r/pp1qn1pp/2nbpp2/2pp4/5N2/4P3/PPPPQPPP/RN...  -65.945526
1            1Q6/2R5/4pk2/6q1/P7/1K6/8/7q b - - 0 42  489.331116
2  r4rk1/pp1b1qpp/3Pp2n/2B1p3/8/P5NP/1P2QPP1/2R1K... -421.389404
3                8/8/4k3/pKp2r2/2P5/8/8/8 b - - 3 53  543.987671
4  q3kbnr/1b1p1ppp/4p3/1p2n3/2p1P3/2PP4/1P2QPPP/1...   16.349640
5  rnbqk2r/pppp1ppp/5n2/4p3/2B1P3/2N2P2/PPPP2PP/R...  -26.700813
6  1r4k1/4q1p1/3p1p1p/prpPpP1Q/8/P3P2P/1P1R1RP1/6...   52.096733
7  7r/p1k4P/1p6/2p1b1P1/P1P5/4PB2/5PK1/7R w - - 1 41  550.215576
8       8/1p1N4/1p4p1/1P1P3k/1r6/p3K3/8/8 w - - 2 49 -500.272095
9  r4br1/ppk2p2/2p2p1p/6p1/4N3/P4P1P/1PP2P2/3R1RK...  175.039429



                                                  ID  Evaluation
0  r3k2r/pp1qn1pp/2nbpp2/2pp4/5N2/4P3/PPPPQPPP/RN...  -65.945526
1            1Q6/2R5/4pk2/6q1/P7/1K6/8/7q b - - 0 42 -489.331116
2  r4rk1/pp1b1qpp/3Pp2n/2B1p3/8/P5NP/1P2QPP1/2R1K...  421.389404
3                8/8/4